# Write Billboard Top 100

This idea of this notebook is to check the most recent date in the output file, and write all the charts to the present date.

> This is very hacky at the moment. I should also be updated with the time-out steps in build-archive (which are not perfect).

The update will time out if there is somewhere around 10+ weeks. There could be a major problem if it times out before it gets through its counter, as in it could get the most recent week, but not some gap weeks. The next running would not fill the gaps. Not quite sure what to do about that at this point.

So, save the repo before updating. Maybe do a check at the end.

In [1]:
import billboard
from datetime import datetime, timedelta, date
import os
import pandas as pd

## Settings

Some variables that don't typically change but could.

In [2]:
outfilename = '../data/output.csv'
chart_type = 'hot-100'
headers = 'date,title,artist,current,previous,peak,weeks\n'

## Create file if necessary

Checks for file. Creates if not there. Adds headers if empty.

In [3]:
file_exists = os.path.exists(outfilename)

# checks if file exists and writes if not
if file_exists != True:
    with open(outfilename, 'a') as outputfile:
        outputfile.write(headers)
        print("File created with header")
# checks if file empty and writes header if not
else:
    file_empty = os.stat(outfilename).st_size == 0
    if file_empty:
        with open(outfilename, 'a') as outputfile:
            outputfile.write(headers)
            print("Added header")
    else:
        print("File has data")

File has data


## Check most recent date of output

> This portion will error if there is no date in the file.

- Import the file
- Get max of date
- Figure out how many weeks to current week
- Billboard comes out on Saturday, which is day 6 in python
- Set counter to number of weeks to current date
- Then the script should pull the number of weeks from max saturday


In [5]:
def weeks_diff(start, end):
    """
    Find difference between last date in chart
    and current chart date
    """
    days = pd.to_datetime(end) - pd.to_datetime(start)
    week = int(pd.Timedelta(days).days / 7)
    remainder = pd.Timedelta(days).days % 7
    return week

# read in file
top_100 = pd.read_csv(outfilename)

# find most recent week in output
most_recent_date = top_100.date.max()
print("Most recent date in file: " + most_recent_date)

# find current chart date
current_chart_date = billboard.ChartData(chart_type).date
print("Current chart date is: " + current_chart_date)

# weeks from last update
weeks_since_update = weeks_diff(most_recent_date, current_chart_date)
print("Weeks since last update: " + str(weeks_since_update))

# set counter based on weeks to update
counter = weeks_since_update
print("Weeks to pull: " + str(counter))


Most recent date in file: 2020-01-18
Current chart date is: 2020-01-18
Weeks since last update: 0
Weeks to pull: 0


## Writing the files

This writes the file based on the `counter` and `chart_date` above. I have to figure out how to check the most recent date and the pull new lists.

In [6]:
# counter = 2 # number of charts to be pulled at a time
# start_date = '2019-12-01'
chart_date = current_chart_date
chart = billboard.ChartData(chart_type, date=chart_date)

with open(outfilename, 'a') as outputfile:
    for i in range (1,counter+1):
        for position in range (0,100):
            song = chart[position]
            line_out = str(chart.date) + ',' + '"' + song.title + '"' + ',' + '"' \
            + song.artist + '"' + ','  + str(song.rank) + ',' + str(song.lastPos) \
            + ',' + str(song.peakPos) + ',' + str(song.weeks) + '\n'
            with open(outfilename, 'a') as outputfile:
                outputfile.write(line_out)
        print(chart.previousDate)
        print(chart[0])
        chart_date = chart.previousDate
        chart = billboard.ChartData(chart_type, str(chart.previousDate))
    print('done')

done


## Checking recent entries

Check through the most recent dates to make sure there aren't any gaps. If there are, I'm not sure what to do.

In [7]:
chart_update = pd.read_csv(outfilename)

grouped = chart_update.groupby(['date']).count().sort_values('date', ascending=False)

grouped.head(25)

,title,artist,current,previous,peak,weeks
date,,,,,,
2020-01-18,100,100,100,100,100,100
2020-01-11,100,100,100,100,100,100
2020-01-04,100,100,100,100,100,100
2019-12-28,100,100,100,100,100,100
2019-12-21,100,100,100,100,100,100
2019-12-14,100,100,100,100,100,100
2019-12-07,100,100,100,100,100,100
